In [1]:
import pandas as pd

df=pd.read_csv(r"customers-data - customers-data.csv")

In [2]:
df.head()

,customer_id,products_purchased,complains,money_spent
0,1000661,1,0,260.0
1,1001914,1,0,790.2
2,1002167,3,0,234.2
3,1002387,1,0,890.0
4,1002419,2,0,103.0


In [3]:
df.shape

(24000, 4)

In [4]:
df.describe()

,customer_id,products_purchased,complains,money_spent
count,2.400000e+04,24000.000000,24000.000000,24000.000000
mean,1.001908e+06,1.600000,0.010000,456.830000
std,6.489202e+02,0.800017,0.117901,320.704042
min,1.000660e+06,1.000000,0.000000,102.000000
25%,1.001913e+06,1.000000,0.000000,233.700000
50%,1.002166e+06,1.000000,0.000000,260.500000
75%,1.002387e+06,2.000000,0.000000,793.200000
max,1.002419e+06,3.000000,2.000000,894.000000


In [5]:
import numpy as np
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [6]:
# Define K-means model
kmeans_model = KMeans(init='k-means++',  max_iter=400, random_state=42)

In [7]:
# Train the model
kmeans_model.fit(df[['products_purchased','complains',
'money_spent']])

KMeans(max_iter=400, random_state=42)

In [8]:
#Create the K means model for different values of K
def try_different_clusters(K, data):

    cluster_values = list(range(1, K+1))
    inertias=[]

    for c in cluster_values:
        model = KMeans(n_clusters = c,init='k-means++',max_iter=400,random_state=42)
        model.fit(data)
        inertias.append(model.inertia_)

    return inertias

In [9]:
# Find output for k values between 1 to 12 
outputs = try_different_clusters(12, df[['products_purchased','complains','money_spent']])
distances = pd.DataFrame({"clusters": list(range(1, 13)),"sum of squared distances": outputs})

In [10]:
# Finding optimal number of clusters k
figure = go.Figure()
figure.add_trace(go.Scatter(x=distances["clusters"], y=distances["sum of squared distances"]))

figure.update_layout(xaxis = dict(tick0 = 1,dtick = 1,tickmode = 'linear'),
                  xaxis_title="Number of clusters",
                  yaxis_title="Sum of squared distances",
                  title_text="Finding optimal number of clusters using elbow method")
figure.show()

In [11]:
# Re-Train K means model with k=2
kmeans_model_new = KMeans(n_clusters = 2,init='k-means++',max_iter=400,random_state=42)

kmeans_model_new.fit_predict(df[['products_purchased','complains','money_spent']])

array([1, 0, 1, ..., 1, 0, 1], dtype=int32)

In [12]:
# Create data arrays
cluster_centers = kmeans_model_new.cluster_centers_
data = np.expm1(cluster_centers)
points = np.append(data, cluster_centers, axis=1)
points

C:\Users\yuvra\AppData\Local\Temp\ipykernel_11552\3873240996.py:3: RuntimeWarning:

overflow encountered in expm1



array([[1.71828183e+00, 1.51130646e-02,            inf, 1.00000000e+00,
        1.50000000e-02, 8.42225000e+02],
       [6.38905610e+00, 6.68893835e-03, 6.53833145e+86, 2.00000000e+00,
        6.66666667e-03, 1.99900000e+02]])

In [13]:
# Add "clusters" to customers data
points = np.append(points, [[0], [1]], axis=1)
df["clusters"] = kmeans_model_new.labels_

In [14]:
# visualize clusters
figure = px.scatter_3d(df,
                    color='clusters',
                    x="products_purchased",
                    y="complains",
                    z="money_spent",
                    category_orders = {"clusters": ["0", "1"]}
                    )
figure.update_layout()
figure.show()